In [411]:
import pandas as pd
import numpy as np
import pickle

In [412]:
dataset=pd.read_csv("computer_dataset_clean.csv")

In [413]:
dataset.isna().sum()
dataset["Weight"].fillna("0.0",inplace=True)
dataset.isna().sum()

Unnamed: 0          0
Company             0
TypeName            0
Inches              0
ScreenResolution    0
Cpu                 0
Ram                 0
Memory              0
Gpu                 0
OpSys               0
Weight              0
Price               0
dtype: int64

In [414]:
dataset.shape

(1243, 12)

# 1 drop categorical data

In [415]:
lis_col=['Unnamed: 0', 'Company', 'TypeName', 'ScreenResolution',
       'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight']
lis_col

['Unnamed: 0',
 'Company',
 'TypeName',
 'ScreenResolution',
 'Cpu',
 'Ram',
 'Memory',
 'Gpu',
 'OpSys',
 'Weight']

In [416]:

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

for col in lis_col:  # Replace with actual column names
    dataset[col] = encoder.fit_transform(dataset[col])

pickle.dump(encoder,open("coding","wb"))

In [417]:
dataset

,Unnamed: 0,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,0,1,4,13.3,23,65,9,4,58,8,40,71378.6832
1,1,1,4,13.3,1,63,9,2,51,8,37,47895.5232
2,2,7,3,15.6,8,74,9,16,53,4,76,30636.0000
3,3,1,4,15.4,25,85,1,29,9,8,73,135195.3360
4,4,1,4,13.3,23,67,9,16,59,8,40,96095.8080
...,...,...,...,...,...,...,...,...,...,...,...,...
1238,1238,2,3,15.6,0,89,6,26,65,5,116,38378.6496
1239,1239,10,0,14.0,13,89,6,4,47,5,80,33992.6400
1240,1240,10,0,13.3,19,89,1,29,47,5,43,79866.7200
1241,1241,10,3,14.0,0,34,4,35,40,5,58,12201.1200


In [418]:
independent=dataset.drop("Price",axis=1)
dependent=dataset["Price"]

# 2 we r going to select the best features

In [419]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
def selectk(n,independent,dependent):
    model=SelectKBest(k=n,score_func=f_regression)
    model.fit(independent,dependent)
    selected_feature_names = independent.columns[model.get_support()]
    selected_features=model.transform(independent)
    return selected_features,selected_feature_names 


In [420]:
kbest,selected_feature_names=selectk(5,independent,dependent)

In [421]:
kbest

array([[23., 65.,  9., 58.,  8.],
       [ 1., 63.,  9., 51.,  8.],
       [ 8., 74.,  9., 53.,  4.],
       ...,
       [19., 89.,  1., 47.,  5.],
       [ 0., 34.,  4., 40.,  5.],
       [ 0., 89.,  8., 21.,  5.]])

In [422]:
selected_feature_names

Index(['ScreenResolution', 'Cpu', 'Ram', 'Gpu', 'OpSys'], dtype='object')

# 3 splitting and standardisation

In [423]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
def split_scaler(independent,dependent):
    
    x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=False)
    sc=StandardScaler()
    x_train=sc.fit_transform(x_train)
    x_test=sc.transform(x_test)
    return x_train,x_test,y_train,y_test,sc

In [424]:
x_train,x_test,y_train,y_test,SC=split_scaler(kbest,dependent)

In [425]:
SC
pickle.dump(SC,open("scx","wb"))

# 4 now  we r going to create models to see the model performance with kbest features

In [426]:

accsvm=[]
accs_MULTILINEAR=[]
accRF=[]
accDEC_tree=[]



In [427]:
def r2_score(model,x_test,y_test):
    from sklearn.metrics import r2_score
    y_predict=model.predict(x_test)
    r_score=r2_score(y_test,y_predict)
    return r_score,model
    

In [428]:
def lin_reg(x_train,y_train,x_test,y_test):
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import GridSearchCV
    model=LinearRegression()
    param_grid={"fit_intercept":[True,False]}
    grid_Search= GridSearchCV(model,param_grid,verbose=3,n_jobs=1,refit=True)
    grid_Search.fit(x_train,y_train)
    r_score,model=r2_score(grid_Search,x_test,y_test)
    return r_score,model

In [429]:
R2_score,model=lin_reg(x_train,y_train,x_test,y_test)
accs_MULTILINEAR.append(R2_score)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END ................fit_intercept=True;, score=0.489 total time=   0.0s
[CV 2/5] END ................fit_intercept=True;, score=0.451 total time=   0.0s
[CV 3/5] END ................fit_intercept=True;, score=0.450 total time=   0.0s
[CV 4/5] END ................fit_intercept=True;, score=0.445 total time=   0.0s
[CV 5/5] END ................fit_intercept=True;, score=0.465 total time=   0.0s
[CV 1/5] END ..............fit_intercept=False;, score=-2.628 total time=   0.0s
[CV 2/5] END ..............fit_intercept=False;, score=-2.849 total time=   0.0s
[CV 3/5] END ..............fit_intercept=False;, score=-2.998 total time=   0.0s
[CV 4/5] END ..............fit_intercept=False;, score=-3.564 total time=   0.0s
[CV 5/5] END ..............fit_intercept=False;, score=-2.849 total time=   0.0s


In [430]:
accs_MULTILINEAR

[0.475048248166564]

In [431]:
def svm(x_train,y_train,x_test,y_test):
    from sklearn.svm import SVR
    from sklearn.model_selection import GridSearchCV
    model=SVR()
    param_grid={"kernel":["poly","sigmoid","rbf"],"C":[0.1,1,10]}
    grid_search = GridSearchCV(model,param_grid,verbose=3,n_jobs=1,refit=True)
    grid_search.fit(x_train,y_train)
    R2_score,model=r2_score(grid_search,x_test,y_test)
    return R2_score,model

In [432]:
R2_score,model=svm(x_train,y_train,x_test,y_test)
accsvm.append(R2_score)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END ...............C=0.1, kernel=poly;, score=-0.027 total time=   0.0s
[CV 2/5] END ...............C=0.1, kernel=poly;, score=-0.054 total time=   0.0s
[CV 3/5] END ...............C=0.1, kernel=poly;, score=-0.026 total time=   0.0s
[CV 4/5] END ...............C=0.1, kernel=poly;, score=-0.033 total time=   0.0s
[CV 5/5] END ...............C=0.1, kernel=poly;, score=-0.035 total time=   0.0s
[CV 1/5] END ............C=0.1, kernel=sigmoid;, score=-0.028 total time=   0.0s
[CV 2/5] END ............C=0.1, kernel=sigmoid;, score=-0.054 total time=   0.0s
[CV 3/5] END ............C=0.1, kernel=sigmoid;, score=-0.027 total time=   0.0s
[CV 4/5] END ............C=0.1, kernel=sigmoid;, score=-0.033 total time=   0.0s
[CV 5/5] END ............C=0.1, kernel=sigmoid;, score=-0.035 total time=   0.0s
[CV 1/5] END ................C=0.1, kernel=rbf;, score=-0.028 total time=   0.0s
[CV 2/5] END ................C=0.1, kernel=rbf;, 

In [433]:
accsvm

[0.00811201111132931]

In [434]:
from sklearn.tree import DecisionTreeRegressor

In [435]:
def decision_tree(x_train,y_train,x_test,y_test):
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.model_selection import GridSearchCV
    model=DecisionTreeRegressor()
    param_grid={"criterion":["friedman_mse","squared_error"],"splitter":["random","best"],"max_depth":[None,5,10]}
    grid_search=GridSearchCV(model,param_grid,verbose=3,n_jobs=1,refit=True)
    grid_search.fit(x_train,y_train)
    R2_score,model=r2_score(grid_search,x_test,y_test)
    return R2_score,model
    

In [436]:
R2_score,model=decision_tree(x_train,y_train,x_test,y_test)
accDEC_tree.append(R2_score)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END criterion=friedman_mse, max_depth=None, splitter=random;, score=0.599 total time=   0.0s
[CV 2/5] END criterion=friedman_mse, max_depth=None, splitter=random;, score=0.652 total time=   0.0s
[CV 3/5] END criterion=friedman_mse, max_depth=None, splitter=random;, score=0.516 total time=   0.0s
[CV 4/5] END criterion=friedman_mse, max_depth=None, splitter=random;, score=0.594 total time=   0.0s
[CV 5/5] END criterion=friedman_mse, max_depth=None, splitter=random;, score=0.576 total time=   0.0s
[CV 1/5] END criterion=friedman_mse, max_depth=None, splitter=best;, score=0.612 total time=   0.0s
[CV 2/5] END criterion=friedman_mse, max_depth=None, splitter=best;, score=0.584 total time=   0.0s
[CV 3/5] END criterion=friedman_mse, max_depth=None, splitter=best;, score=0.554 total time=   0.0s
[CV 4/5] END criterion=friedman_mse, max_depth=None, splitter=best;, score=0.601 total time=   0.0s
[CV 5/5] END criterion=friedm

In [437]:
accDEC_tree

[0.6340542768359769]

In [438]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [476]:
def random_forest(x_train,y_train,x_test,y_test):
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.model_selection import GridSearchCV
    model=RandomForestRegressor()
    param_grid={"n_estimators": [100, 200, 500],"criterion":["friedman_mse","squared_error"],"max_depth":[None,5,10]}
    grid_search=GridSearchCV(model,param_grid,verbose=3,n_jobs=1,refit=True)
    grid_search.fit(x_train,y_train)
    fin_model=grid_search.best_estimator_ 
    R2_score,model=r2_score(grid_search,x_test,y_test)
    return R2_score,fin_model

In [477]:
R2_score,model=random_forest(x_train,y_train,x_test,y_test)
accRF.append(R2_score)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END criterion=friedman_mse, max_depth=None, n_estimators=100;, score=0.673 total time=   0.3s
[CV 2/5] END criterion=friedman_mse, max_depth=None, n_estimators=100;, score=0.686 total time=   0.2s
[CV 3/5] END criterion=friedman_mse, max_depth=None, n_estimators=100;, score=0.688 total time=   0.2s
[CV 4/5] END criterion=friedman_mse, max_depth=None, n_estimators=100;, score=0.723 total time=   0.2s
[CV 5/5] END criterion=friedman_mse, max_depth=None, n_estimators=100;, score=0.697 total time=   0.2s
[CV 1/5] END criterion=friedman_mse, max_depth=None, n_estimators=200;, score=0.675 total time=   0.6s
[CV 2/5] END criterion=friedman_mse, max_depth=None, n_estimators=200;, score=0.689 total time=   0.6s
[CV 3/5] END criterion=friedman_mse, max_depth=None, n_estimators=200;, score=0.696 total time=   0.6s
[CV 4/5] END criterion=friedman_mse, max_depth=None, n_estimators=200;, score=0.716 total time=   0.6s
[CV 5/5] END

[CV 1/5] END criterion=squared_error, max_depth=10, n_estimators=200;, score=0.668 total time=   0.5s
[CV 2/5] END criterion=squared_error, max_depth=10, n_estimators=200;, score=0.693 total time=   0.5s
[CV 3/5] END criterion=squared_error, max_depth=10, n_estimators=200;, score=0.704 total time=   0.6s
[CV 4/5] END criterion=squared_error, max_depth=10, n_estimators=200;, score=0.715 total time=   0.5s
[CV 5/5] END criterion=squared_error, max_depth=10, n_estimators=200;, score=0.700 total time=   0.5s
[CV 1/5] END criterion=squared_error, max_depth=10, n_estimators=500;, score=0.670 total time=   1.4s
[CV 2/5] END criterion=squared_error, max_depth=10, n_estimators=500;, score=0.688 total time=   1.4s
[CV 3/5] END criterion=squared_error, max_depth=10, n_estimators=500;, score=0.700 total time=   1.4s
[CV 4/5] END criterion=squared_error, max_depth=10, n_estimators=500;, score=0.721 total time=   1.4s
[CV 5/5] END criterion=squared_error, max_depth=10, n_estimators=500;, score=0.703

In [478]:
model

RandomForestRegressor(n_estimators=200)

In [464]:
accRF

[0.778027728215024, 0.7775686855920535]

In [443]:
def selectk_classification(accs_MULTILINEAR,accsvm,accDEC_tree,accRF):
    dataframe=pd.DataFrame(index=["f_regression"],columns=["multilinear","svm","decisiontree","RF"])
    for number,idex in enumerate(dataframe.index):
                           dataframe["multilinear"][idex]=accs_MULTILINEAR[number]
                           dataframe["svm"][idex]=accsvm[number]
                           dataframe["decisiontree"][idex]=accDEC_tree[number]
                           dataframe["RF"][idex]=accRF[number]
    return dataframe
   

In [444]:
result1=selectk_classification(accs_MULTILINEAR,accsvm,accDEC_tree,accRF)

In [445]:
result1# result1 for 5

,multilinear,svm,decisiontree,RF
f_regression,0.475048,0.008112,0.634054,0.778028


In [322]:
result1# result1 for 7

,multilinear,svm,decisiontree,RF
f_regression,0.498917,-0.028345,0.614015,0.759221


In [345]:
result1# result1 for 8

,multilinear,svm,decisiontree,RF
f_regression,0.498614,-0.035742,0.69088,0.776141


In [368]:
result1# for 4

,multilinear,svm,decisiontree,RF
f_regression,0.42955,0.021915,0.617334,0.706621


In [480]:
final_model=model

In [481]:
pickle.dump(final_model,open("model","wb"))

In [482]:
model=pickle.load(open("model","rb"))

In [483]:
model

RandomForestRegressor(n_estimators=200)

In [484]:
selected_feature_names

Index(['ScreenResolution', 'Cpu', 'Ram', 'Gpu', 'OpSys'], dtype='object')

In [485]:
independent=[["1440x900","Intel Core i5 2.3GHz","8GB","Intel Iris Plus Graphics 640","macOS"]]

In [486]:
input_table=pd.DataFrame(independent,columns=selected_feature_names)

In [487]:
scx=pickle.load(open("scx","rb"))
encode=pickle.load(open("coding","rb"))

In [488]:
for col in input_table:
    input_table[col]=encode.fit_transform(input_table[col])

In [489]:
input_table

,ScreenResolution,Cpu,Ram,Gpu,OpSys
0,0,0,0,0,0


In [490]:
final_input=scx.transform(input_table)

C:\Users\nimmy\anaconda3\envs\aiml\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [491]:
model.predict(final_input)

array([51549.32313943])